# Content-based recommender system for movies

In [1]:
#pip install rake_nltk


In [2]:
#pip install --upgrade pandas


In [3]:
#pip install sklearn

In [4]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

#pd.set_option('display.max_columns', 100)


In [5]:
df = pd.read_csv('final_data(movies 1950-2020).csv')
df.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,tmdb_ratings
0,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,insidious: the last key,6.2
1,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,the strange ones,5.5
2,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,stratton,4.9
3,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Drama History Western,sweet country,6.7
4,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller,the commuter,6.3


In [6]:
df.shape

(6095, 7)

Choosing the features that will be base of my recommendations.

In [7]:
df = df[['movie_title','tmdb_ratings','director_name','actor_1_name','actor_2_name','actor_3_name','genres']] 
df.head()

,movie_title,tmdb_ratings,director_name,actor_1_name,actor_2_name,actor_3_name,genres
0,insidious: the last key,6.2,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller
1,the strange ones,5.5,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama
2,stratton,4.9,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller
3,sweet country,6.7,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Drama History Western
4,the commuter,6.3,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller


In [8]:
df.head()

,movie_title,tmdb_ratings,director_name,actor_1_name,actor_2_name,actor_3_name,genres
0,insidious: the last key,6.2,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller
1,the strange ones,5.5,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama
2,stratton,4.9,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller
3,sweet country,6.7,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Drama History Western
4,the commuter,6.3,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller


In [9]:
df.isnull().sum()

movie_title       0
tmdb_ratings     97
director_name     0
actor_1_name      0
actor_2_name      0
actor_3_name      0
genres            3
dtype: int64

In [10]:
df['genres'] = df['genres'].fillna('unknown')

In [11]:
df.isnull().sum()

movie_title       0
tmdb_ratings     97
director_name     0
actor_1_name      0
actor_2_name      0
actor_3_name      0
genres            0
dtype: int64

In [12]:
df['comb'] = df['actor_1_name'] + ' ' + df['actor_2_name'] + ' '+ df['actor_3_name'] + ' '+ df['director_name'] +' ' + df['genres']

In [13]:
(df['movie_title']=='tenet').sum()

1

In [14]:
(df['movie_title'] == 'tenet').sum()

1

Extracting the key words from the plot description.

In [15]:
df.head()

,movie_title,tmdb_ratings,director_name,actor_1_name,actor_2_name,actor_3_name,genres,comb
0,insidious: the last key,6.2,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
1,the strange ones,5.5,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,Alex Pettyfer James Freedson-Jackson Emily Alt...
2,stratton,4.9,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,Dominic Cooper Austin Stowell Gemma Chan Simon...
3,sweet country,6.7,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Drama History Western,Bryan Brown Sam Neill unknown Warwick Thornton...
4,the commuter,6.3,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...


In [16]:
df.set_index('movie_title', inplace = True)

df.head()

,tmdb_ratings,director_name,actor_1_name,actor_2_name,actor_3_name,genres,comb
movie_title,,,,,,,
insidious: the last key,6.2,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
the strange ones,5.5,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,Alex Pettyfer James Freedson-Jackson Emily Alt...
stratton,4.9,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,Dominic Cooper Austin Stowell Gemma Chan Simon...
sweet country,6.7,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Drama History Western,Bryan Brown Sam Neill unknown Warwick Thornton...
the commuter,6.3,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...


In [17]:
df

,tmdb_ratings,director_name,actor_1_name,actor_2_name,actor_3_name,genres,comb
movie_title,,,,,,,
insidious: the last key,6.2,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
the strange ones,5.5,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,Alex Pettyfer James Freedson-Jackson Emily Alt...
stratton,4.9,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,Dominic Cooper Austin Stowell Gemma Chan Simon...
sweet country,6.7,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Drama History Western,Bryan Brown Sam Neill unknown Warwick Thornton...
the commuter,6.3,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...
...,...,...,...,...,...,...,...
news of the world,NaN,Paul Greengrass,Tom Hanks,Helena Zengel,unknown,Drama Western,Tom Hanks Helena Zengel unknown Paul Greengras...
one night in miami,NaN,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
promising young woman,5.8,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [18]:
df.head()

,tmdb_ratings,director_name,actor_1_name,actor_2_name,actor_3_name,genres,comb
movie_title,,,,,,,
insidious: the last key,6.2,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Horror Mystery Thriller,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
the strange ones,5.5,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Thriller Drama,Alex Pettyfer James Freedson-Jackson Emily Alt...
stratton,4.9,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Action Thriller,Dominic Cooper Austin Stowell Gemma Chan Simon...
sweet country,6.7,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Drama History Western,Bryan Brown Sam Neill unknown Warwick Thornton...
the commuter,6.3,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Action Thriller,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...


In [19]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['comb'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(df.index)
indices.head()

0    insidious: the last key
1           the strange ones
2                   stratton
3              sweet country
4               the commuter
Name: movie_title, dtype: object

In [20]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix)
cosine_sim

array([[1.        , 0.09090909, 0.09534626, ..., 0.09090909, 0.        ,
        0.        ],
       [0.09090909, 1.        , 0.09534626, ..., 0.18181818, 0.09534626,
        0.10050378],
       [0.09534626, 0.09534626, 1.        , ..., 0.09534626, 0.        ,
        0.        ],
       ...,
       [0.09090909, 0.18181818, 0.09534626, ..., 1.        , 0.09534626,
        0.10050378],
       [0.        , 0.09534626, 0.        , ..., 0.09534626, 1.        ,
        0.10540926],
       [0.        , 0.10050378, 0.        , ..., 0.10050378, 0.10540926,
        1.        ]])

In [21]:
# function that takes in movie_title as input and returns the top 10 recommended movies

def recommendations(title):
    
    recommended_movies = []
    ratings = []
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        ratings.append(list(df['tmdb_ratings'])[i-1])
    return recommended_movies,ratings

In [29]:
recommendations('avatar') 

(['aliens',
  'mystery men',
  'deep rising',
  'dragonball: evolution',
  'the terminator',
  'terminator 2: judgment day',
  'blade: trinity',
  'tron: legacy',
  'dune',
  'guardians of the galaxy'],
 [7.6, 6.9, 6.8, 5.3, 7.6, 7.1, 6.1, 6.4, 5.7, 6.1])

In [32]:
 
x = pd.DataFrame()
x[' Recommended movies'] = recommendations('the nun')[0]
x['ratings'] = recommendations('the nun')[1]

In [33]:
x

,Recommended movies,ratings
0,the grudge,4.3
1,run,6.4
2,prey,6.8
3,annabelle comes home,6.6
4,the skeleton key,6.8
5,polaroid,6.4
6,an american haunting,4.6
7,urban legend,6.8
8,case 39,6.1
9,savages,6.3


In [25]:
 rcmd_movies = pd.DataFrame()

In [26]:
# function that takes in genres values as input and returns the top 10 recommended movies

def recomd(title):
    
    recommended_movies = []
    ratings = []
    title=title.lower()
    df1=np.array(df['comb'])
    df1=np.append(title,df1)
    
    idx = 0
    count1 = CountVectorizer()
    count_matrix = count.fit_transform(df1)
    c2=cosine_similarity(count_matrix)
    
    score_series = pd.Series(c2[0]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i-1])
        ratings.append(list(df['tmdb_ratings'])[i-1])
    
        
    return recommended_movies,ratings

In [27]:
x = pd.DataFrame()
x['movie']=recomd('drama')[0]
x['ratings']=recomd('drama')[1]

In [28]:
x

,movie,ratings
0,the assistant,6.6
1,wendy and lucy,6.7
2,labor day,6.9
3,an american in hollywood,NaN
4,derailed,6.3
5,the namesake,6.7
6,blue car,6.3
7,kavan,7.1
8,soundtrack,6.5
9,old joy,6.5
